In [1]:
from PIL import Image
import numpy as np
from mtcnn.mtcnn import MTCNN 
from pathlib import Path

In [2]:
def extract_face(path, detector=MTCNN(), required_size=(160,160), save_faces=True):
  # load image from file
  image = Image.open(path)

  # convert to rgb
  image = image.convert('RGB')

  # covert to array
  pixels = np.asarray(image)

  #detect faces in the image
  results = detector.detect_faces(pixels)

  # extract the bounding box from the first face
  x1, y1, width, height = results[0]['box']

  # bug fix
  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height

  # extract the face
  face = pixels[y1:y2, x1:x2]

  # resize pixels to the model size
  image = Image.fromarray(face)
  image = image.resize(required_size)

  face_array = np.asarray(image)
  return face_array

In [ ]:
# we can save X and Y into a zip file and load it:
# np.savez_compressed('FileName.npz', X's (faces as np array), Y's (labels))
# we can load them using :
# data = np.load(path of .npz)
# X, Y = data['arr_0'], data['arr_1']

In [3]:
import tensorflow as tf

In [ ]:
def get_embedding(model, face_pixels):
  # scale pixel values
  face_pixels = face_pixels.astype('float32')

  # normalization
  mean, std = face_pixels.mean(), face_pixels.std()
  face_pixels = (face_pixels - mean) / std
  
  # transform face into one sample
  samples = np.expand_dims(face_pixels, axis=0)

  # make predictions to get embeddings
  yhat = model.predict(samples)

  return yhat[0]

In [ ]:
# load the photos
# data = np.load(path of .npz)
# X, Y = data['arr_0'], data['arr_1']

In [ ]:
# load the facenet model
model = tf.keras.models.load_model('facenet_keras.h5')

In [ ]:
# get the embeddings of all pics
# img_pixels = extract_face('man.jpg')
# embeddings = get_embedding(model, img_pixels)
# make the embeddings np array
# save them using np.savez_compressed(name.npz, embeddings np array, labels)

In [ ]:
# import keras model
# model = tf.keras.models.load_model('facenet_keras.h5')

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
def predict_using_distance(faces_embeddings, labels, face_to_predict_embeddings):
  # normalize input vector
  in_encoder = Normalizer(norm='l2')
  faces_embeddings = in_encoder.transform(faces_embeddings)
  face_to_predict_embeddings = in_encoder.transform(face_to_predict_embeddings)

  # use euclidean distance
  # the distance gives how similar the faces are
  face_distance = np.linalg.norm(faces_embeddings - face_to_predict_embeddings, axis=1)

  name = 'Unknown'

  # put threshold for the distance to know if the person is found or not
  threshold = 0.7

  # list of matching people
  matching = []
  for i in range(len(face_distance)):
    if(face_distance[i] < threshold):
      matching.append([face_distance[i], labels[i]])
  
  min_label = 'Unknown'
  min_dist = 213124
  for i in range(len(matching)):
    if(matching[i][0] < min_dist):
      min_dist = matching[i][0]
      min_label = matching[i][1]
    
  return min_label

In [ ]:
# img1 = extract_face('michael-jordan.jpg')
# img2 = extract_face('michael-jordan2.jpg')
# X = [img1, img2]
# Y = np.asarray(['michael-jordan', 'michael-jordan'])

# for i in range(len(X)):
#   X[i] = get_embedding(model, X[i])

In [ ]:
# img3 = extract_face('michael-jordan3.jpg')
# img3_emb = get_embedding(model, img3)

# img4 = extract_face('man.jpg')
# img4_emb = get_embedding(model, img4)

In [ ]:
# img5 = extract_face('michael-jordan-t.jpg')
# img5_emb = get_embedding(model, img5)

In [ ]:
# print(predict_using_distance(np.asarray(X), Y, img5_emb.reshape(1,-1)))